In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexLayers import ComplexBatchNorm2d, ComplexConv2d, ComplexLinear
from complexLayers import ComplexDropout2d, NaiveComplexBatchNorm2d
from complexFunctions import complex_relu, complex_max_pool2d

In [2]:
batch_size = 64
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST('../data', train=True, transform=trans, download=True)
test_set = datasets.MNIST('../data', train=False, transform=trans, download=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle=True)

In [4]:
class ComplexNet(nn.Module):
    
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = ComplexConv2d(1, 10, 5, 1)
        self.bn  = ComplexBatchNorm2d(10)
        self.conv2 = ComplexConv2d(10, 20, 5, 1)
        self.fc1 = ComplexLinear(4*4*20, 500)
        self.dropout = ComplexDropout2d(p = 0.3)
        self.fc2 = ComplexLinear(500, 10)
             
    def forward(self,x):
        x = self.conv1(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = self.bn(x)
        x = self.conv2(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = x.view(-1,4*4*20)
        x = self.fc1(x)
        x = self.dropout(x)
        x = complex_relu(x)
        x = self.fc2(x)
        x = x.abs()
        x =  F.log_softmax(x, dim=1)
        return x
    
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ComplexNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target =data.to(device).type(torch.complex64), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item())
            )

In [ ]:
# Run training on 4 epochs
for epoch in range(4):
    train(model, device, train_loader, optimizer, epoch)

Train Epoch:   0 [     0/ 60000 (  0%)]	Loss: 2.568596


/home/spopoff/dev/complexPyTorch/complexFunctions.py:54: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370124688/work/aten/src/ATen/native/Copy.cpp:162.)
  mask = torch.ones_like(input).type(torch.float32)


Train Epoch:   0 [  6400/ 60000 ( 11%)]	Loss: 0.418056
Train Epoch:   0 [ 12800/ 60000 ( 21%)]	Loss: 0.069222
Train Epoch:   0 [ 19200/ 60000 ( 32%)]	Loss: 0.126281
Train Epoch:   0 [ 25600/ 60000 ( 43%)]	Loss: 0.198962
Train Epoch:   0 [ 32000/ 60000 ( 53%)]	Loss: 0.045389
Train Epoch:   0 [ 38400/ 60000 ( 64%)]	Loss: 0.044656
Train Epoch:   0 [ 44800/ 60000 ( 75%)]	Loss: 0.056991
Train Epoch:   0 [ 51200/ 60000 ( 85%)]	Loss: 0.114485
Train Epoch:   0 [ 57600/ 60000 ( 96%)]	Loss: 0.027030
Train Epoch:   1 [     0/ 60000 (  0%)]	Loss: 0.051180
Train Epoch:   1 [  6400/ 60000 ( 11%)]	Loss: 0.026171
Train Epoch:   1 [ 12800/ 60000 ( 21%)]	Loss: 0.072911
Train Epoch:   1 [ 19200/ 60000 ( 32%)]	Loss: 0.008332
Train Epoch:   1 [ 25600/ 60000 ( 43%)]	Loss: 0.047245
Train Epoch:   1 [ 32000/ 60000 ( 53%)]	Loss: 0.150064
Train Epoch:   1 [ 38400/ 60000 ( 64%)]	Loss: 0.129930
Train Epoch:   1 [ 44800/ 60000 ( 75%)]	Loss: 0.029466
Train Epoch:   1 [ 51200/ 60000 ( 85%)]	Loss: 0.034150
Train Epoc